## 1 - Imports and nltk downloads

In [ ]:
import os
download_dir = f'{os.getcwd()}/nltk_data'
os.environ['NLTK_DATA'] = download_dir

import nltk
nltk.download('stopwords', download_dir=download_dir)
nltk.download('punkt', download_dir=download_dir)
nltk.download('wordnet', download_dir=download_dir)
nltk.download('averaged_perceptron_tagger', download_dir=download_dir)
nltk.download('universal_tagset', download_dir=download_dir)
nltk.download('words', download_dir=download_dir)
nltk.data.path.append(download_dir)
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk import ngrams
import spellwise
from spellwise import Levenshtein

import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
import plotly.express as px
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import lightgbm as lgb
from math import sqrt

from functools import partial, reduce
from operator import or_
from pathlib import Path
from pprint import pprint
from typing import Optional, Union, List, Tuple, Dict, Set

## 2 - Fix nltk installation

In [ ]:
%%sh

yes | unzip -q nltk_data/corpora/wordnet.zip -d nltk_data/corpora/
ls nltk_data/corpora/

## 3 - Set data paths

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatiser = WordNetLemmatizer()
tqdm.pandas()

data_dir = Path('../data/commonlit-evaluate-student-summaries')

sample_submission = data_dir / 'sample_submission.csv'
summaries_train = data_dir / 'summaries_train.csv'
summaries_test = data_dir / 'summaries_test.csv'
prompts_train = data_dir / 'prompts_train.csv'
prompts_test = data_dir / 'prompts_test.csv'

content_model = '../data/models/content.txt'
wording_model = '../data/models/wording.txt'

## 4 - Preprocessing functions

In [ ]:
def make_split(summaries_path: Path, prompts_path: Path, dtype_backend: Optional[str] = 'pyarrow') -> pd.DataFrame:
    summaries_df = pd.read_csv(summaries_path)#, dtype_backend=dtype_backend)
    prompts_df = pd.read_csv(prompts_path)#, dtype_backend=dtype_backend)
    df = pd.merge(summaries_df, prompts_df, how='left', on='prompt_id')
    df.fillna('')
    
    return df


def tokenize(text: str) -> List[str]:
    return [lemmatiser.lemmatize(tok.lower()) for tok in word_tokenize(text) if tok.isalnum() and tok not in stop_words]


def make_bigram(tokens: List[str]) -> Set[str]:
    return set(ngrams(tokens, 2))


def clear_stopwords(column: pd.Series, idx: int) -> Union[List[str], List[str], List[str]]:
    tokens = [tok.lower() for tok in word_tokenize(column.iloc[idx]) if tok.isalnum()]
    cleared_stopwords = [tok for tok in tokens if tok not in stop_words]
    lemmas = [lemmatiser.lemmatize(tok) for tok in cleared_stopwords]
    bigram = set(ngrams(lemmas, 2))

    return tokens, cleared_stopwords, lemmas, bigram
    
    
def nlp_preprocess(df: pd.DataFrame, column: str):
    df[f'{column}_lemmas'] = df[column].apply(tokenize)
    df[f'{column}_bigram'] = df[f'{column}_lemmas'].apply(make_bigram)
    
def predict(model: lgb.Booster, df: pd.DataFrame, features: List[str]) -> pd.Series:
    return model.predict(df[features])

## 5 - Load data and preprocess

In [ ]:
# Load data
# df = make_split(summaries_test, prompts_test)
df = make_split(summaries_train, prompts_train)

# Make n-grams for all text columns
text_columns = ['prompt_title', 'prompt_question', 'prompt_text', 'text']
for column in tqdm(text_columns):
    nlp_preprocess(df, column)
    df[f'{column}_unique_bigrams'] = df[f'{column}_bigram'].str.len()

## 6 - Create bigram based features

In [ ]:
df_train = df.copy(deep=True)

# Create n-gram based features
df_train['text_bigram_overlap'] = df_train[['prompt_text_bigram', 'text_bigram']].apply(lambda row: len(row[0] & row[1]), axis=1) / df_train.text_unique_bigrams
df_train['question_bigram_overlap'] = df_train[['prompt_question_bigram', 'text_bigram']].apply(lambda row: len(row[0] & row[1]), axis=1) / df_train.text_unique_bigrams
df_train['text_bigram_ratio'] = df_train['text_unique_bigrams'] / (df_train['prompt_text_unique_bigrams'])

df_train['text_bigram_diff'] = df_train[['prompt_text_bigram', 'text_bigram']].apply(lambda row: len(row[1] - row[0]), axis=1) / df_train.text_unique_bigrams
df_train['question_bigram_diff'] = df_train[['prompt_question_bigram', 'text_bigram']].apply(lambda row: len(row[1] - row[0]), axis=1) / df_train.text_unique_bigrams

df_train['text_bigram_exclusive'] = df_train[['prompt_text_bigram', 'text_bigram']].apply(lambda row: len(row[0] ^ row[1]), axis=1) / df_train.text_unique_bigrams
df_train['question_bigram_exclusive'] = df_train[['prompt_question_bigram', 'text_bigram']].apply(lambda row: len(row[0] ^ row[1]), axis=1) / df_train.text_unique_bigrams

## 7 - Create word based features

In [ ]:
df_train['n_words'] = df_train.text_lemmas.str.len()
df_train['unique_words'] = df_train.text_lemmas.apply(set).str.len()
df_train['unique_ratio'] = df_train.unique_words / df_train.n_words

In [ ]:
df_train['word_lengths'] = df_train.text_lemmas.apply(lambda x: [len(y) for y in x])
df_train['word_len_avg'] = df_train.word_lengths.apply(np.mean)

In [ ]:
df_train['word_len_q10'] = df_train.word_lengths.apply(partial(np.percentile, q=10))
df_train['word_len_q90'] = df_train.word_lengths.apply(partial(np.percentile, q=90))

In [ ]:
x = pos_tag(df_train.text_lemmas[0], tagset='universal')
from collections import defaultdict

dd = defaultdict(lambda: 0)
for _, pos in x:
    dd[pos] += 1

In [ ]:
df_train['pos'] = df_train.text_lemmas.apply(partial(pos_tag, tagset='universal'))

In [ ]:
def pos_counts(tags):
    dd = defaultdict(lambda: 0)
    for _, pos in tags:
        dd[pos] += 1
    return dd

df_train['pos_counts'] = df_train.pos.apply(pos_counts)

In [ ]:
df_train['verb_count'] = df_train.pos_counts.str['VERB'].replace(np.nan, 0)
df_train['noun_count'] = df_train.pos_counts.str['NOUN'].replace(np.nan, 0)
df_train['adv_count'] = df_train.pos_counts.str['ADV'].replace(np.nan, 0)
df_train['adj_count'] = df_train.pos_counts.str['ADJ'].replace(np.nan, 0)
df_train['det_count'] = df_train.pos_counts.str['DET'].replace(np.nan, 0)

In [ ]:
df_train[['verb_count','noun_count','adv_count','adj_count','det_count']].corr()

## 8 - Create spelling based features

In [ ]:
%%sh
wget -nv https://github.com/dwyl/english-words/archive/refs/heads/master.zip -O master.zip
yes | unzip -q master.zip

In [ ]:
with open('./english-words-master/words.txt', 'r') as f:
    en_words = [line.strip() for line in f.read().split('\n')]

en_words = set([word for word in en_words if word.isalpha()])

In [ ]:
def get_unique_words(col: str) -> Set[str]:
    word_sets = df_train[col].apply(set).tolist()
    return reduce(or_, word_sets)

prompt_words = get_unique_words('prompt_text_lemmas')
question_words = get_unique_words('prompt_question_lemmas')
title_words = get_unique_words('prompt_title_lemmas')

word_set = en_words | prompt_words | question_words | title_words

In [ ]:
with open('commonlit_words.txt', 'w') as f:
    f.write('\n'.join(word_set))

In [ ]:
# word_dir = words.abspath('en')
# Initialise the algorithm
metric = Levenshtein()
# Index the words from a dictionary
# metric.add_from_path('./brown_words.txt')
# metric.add_from_path(words.abspath('en'))
# metric.add_from_path('./english-words-master/words.txt')
metric.add_from_path('./commonlit_words.txt')

def get_distances(tokens: List[str], metric: spellwise.algorithms.base.Base) -> List[str]:
    distances = []
    for idx, token in enumerate(tokens):
        suggestions = metric.get_suggestions(token)
        if suggestions == []:
            distance = len(token) if token.isalpha() else 0
        else:
            distance = suggestions[0]['distance']
        distances.append(distance)
    return sum(distances)

In [ ]:
# df_train.text_lemmas.progress_apply(partial(get_distances, metric=metric))

def distance_func(chunk: pd.DataFrame):
    return chunk.apply(partial(get_distances, metric=metric))

n_jobs = 7
df_chunks = np.array_split(df_train.text_lemmas, n_jobs * 2)

total_edit_distances = Parallel(n_jobs=n_jobs, backend='loky')(delayed(distance_func)(chunk) for chunk in tqdm(df_chunks))

In [ ]:
count_missing_words = lambda tokens: sum([word not in word_set for word in tokens])
df_train['missing_wordcount'] = df_train.text_lemmas.progress_apply(count_missing_words)

In [ ]:
numeric_features = df_train.select_dtypes(include=np.number)
target_columns = ['content', 'wording']
feature_columns = [col for col in numeric_features if col not in target_columns]

targets = numeric_features[target_columns]
features = numeric_features[feature_columns]
prompt_group = pd.Categorical(df['prompt_title'])

In [ ]:
def calculate_errors(y, y_pred):
    return {
        'r2': r2_score(y, y_pred),
        'rmse': sqrt(mean_squared_error(y, y_pred)),
        'mae': mean_absolute_error(y, y_pred)
    }


def train_lgb_kfold(
        target: str, 
        prompt_group: pd.DataFrame, 
        features: pd.DataFrame, 
        targets: pd.DataFrame, 
        feature_names: List[str],
        model_params: dict) -> Tuple[pd.DataFrame, lgb.LGBMRegressor]:
    
    group_kfold = GroupKFold(n_splits=prompt_group.unique().size)
    assert group_kfold.get_n_splits(features, targets, prompt_group) == len(prompt_group.unique())

    
    train_errors, val_errors = [], []
    for i, (train_index, test_index) in enumerate(group_kfold.split(features, targets, prompt_group)):
        # print(f'Fold {i}')
        # print(f'\tTest prompt: {df.iloc[test_index].prompt_title.unique().tolist()}')

        X_train = features[feature_names].iloc[train_index].convert_dtypes(dtype_backend='numpy_nullable')
        y_train = targets.iloc[train_index][target].convert_dtypes(dtype_backend='numpy_nullable')

        X_val = features[feature_names].iloc[test_index]
        y_val = targets.iloc[test_index][target]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, y_val)
        bst = lgb.train(model_params, train_data, )#, feval=[r2_score, mean_absolute_error])

        train_errors.append(calculate_errors(y_train, bst.predict(X_train)))
        val_errors.append(calculate_errors(y_val, bst.predict(X_val)))

    train_metrics = pd.DataFrame.from_records(train_errors).describe()
    train_metrics['set'] = 'train'
    val_metrics = pd.DataFrame.from_records(val_errors).describe()
    val_metrics['set'] = 'val'
    metric_df = pd.concat([train_metrics, val_metrics])

    return metric_df, bst

def train_lgb(
        target: str, 
        prompt_group: pd.DataFrame, 
        features: pd.DataFrame, 
        targets: pd.DataFrame, 
        feature_names: List[str],
        model_params: dict) -> Tuple[pd.DataFrame, lgb.LGBMRegressor]:
    
    
    X_train = features[feature_names].convert_dtypes(dtype_backend='numpy_nullable')
    y_train = targets[target].convert_dtypes(dtype_backend='numpy_nullable')


    train_data = lgb.Dataset(X_train, label=y_train)
    bst = lgb.train(model_params, train_data, )#, feval=[r2_score, mean_absolute_error])

    train_errors = [calculate_errors(y_train, bst.predict(X_train))]
    train_metrics = pd.DataFrame.from_records(train_errors)

    return train_metrics, bst

In [ ]:
def eval_validation(f_cols, model_params):
    metric_df_content, bst_content = train_lgb_kfold('content', prompt_group, features, targets, f_cols, model_params)
    metric_df_wording, bst_wording = train_lgb_kfold('wording', prompt_group, features, targets, f_cols, model_params)

    metric_df_content['target'] = 'content'
    metric_df_wording['target'] = 'wording'
    metric_df = pd.concat([metric_df_content, metric_df_wording])
    metric_df = metric_df.loc[['mean', 'std']]
    print(metric_df)

    mcrmse = (metric_df.loc[metric_df.target=='content', 'rmse'] + metric_df.loc[metric_df.target=='wording', 'rmse']) / 2
    print(f'\nTrain MCRMSE:\t   {mcrmse.iloc[0]}')
    print(f'Validation MCRMSE: {mcrmse.iloc[1]}\n')

    importance = pd.DataFrame({
    'importance': bst_wording.feature_importance(),
    'feature': bst_wording.feature_name()}).sort_values(by='importance', ascending=False)
    print(importance)

In [ ]:
features.columns

In [ ]:
model_params = {
    'objective': 'fair', 
    'verbose': 0, 
    'force_col_wise': True,
    'learning_rate': 0.08,
    'boosting_type': 'dart',
    'num_leaves': 11,
}
f_cols = ['text_bigram_overlap', 'text_unique_bigrams', 'unique_ratio', 
          'n_words', 'unique_words', 'word_len_avg', 'word_len_q10', 'word_len_q90',
          'verb_count','noun_count','adv_count','adj_count','det_count', 'missing_wordcount']

eval_validation(f_cols, model_params)

In [ ]:
f_cols = ['text_bigram_overlap', 'text_unique_bigrams', 'unique_ratio', 'missing_wordcount']

eval_validation(f_cols, model_params)

In [ ]:
f_cols = ['text_bigram_overlap', 'text_unique_bigrams', 'missing_wordcount']

eval_validation(f_cols, model_params)

In [ ]:
f_cols = ['text_bigram_overlap', 'text_unique_bigrams', 'unique_ratio', 
          'n_words', 'unique_words', 'word_len_avg', 'word_len_q10', 'word_len_q90',
          'missing_wordcount']

eval_validation(f_cols, model_params)

In [ ]:
metric_df_content, bst_content = train_lgb('content', prompt_group, features, targets, f_cols, model_params)
metric_df_wording, bst_wording = train_lgb('wording', prompt_group, features, targets, f_cols, model_params)

In [ ]:
print(f'\n{"-"*35}\n\tContent scores')
pprint(metric_df_content)
print(f'\n{"-"*35}\n\tWording scores')
pprint(metric_df_wording)

In [ ]:
print('Full data MCRMSE: ')
(metric_df_content.rmse + metric_df_wording.rmse) / 2

In [ ]:
bst_content.save_model(content_model)
bst_wording.save_model(wording_model)